# self-editing memory from scratch (WITHOUT LETTA)

In [1]:
# we need a .env file in the same directory as this code for this to work

from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv()  # load .env as current environment for os to detect

# create OpenAI client
client = OpenAI(
    api_key=str(os.getenv("OPENAI_API_KEY"))
)

model = "gpt-4o-mini"

## understanding the LLM context window

### no memory 🥺

**note:** the roles `system`, `user` and `assistant`are keywords expected by OpenAI. 

In [2]:
system_prompt = "You are a chatbot."

# make a chat completion request
chat_completion = client.chat.completions.create(
    model=model,
    messages=[
        # system prompt: always included in the context window 
        {"role": "system", "content": system_prompt},
        
        # chat history (evolves over time)
        {"role": "user", "content": "What is my name?"},
    ]
)

chat_completion.choices[0].message.content

"I'm sorry, but I don't know your name. You haven't told me yet!"

### faking a conversation 😈

In [3]:
# make a new chat completion request
chat_completion = client.chat.completions.create(
    model=model,
    messages=[
        # system prompt: always included in the context window 
        {"role": "system", "content": system_prompt},
        # NOTE: I REPEAT, THE SYSTEM PROMPT IS ALWAYS INCLUDED IN THE CONTEXT WINDOW!!!
        
        # chat history (evolves over time)
        {"role": "user", "content": "What is my name?"}, 

        {"role": "assistant", "content": "Your name is Sanika. How can I assist you today?"},
        # NOTE: HERE, I TRICKED THE MODEL INTO THINKING IT KNEW MY NAME

        {"role": "user", "content": "Write Haiku about me."},
    ]
)

chat_completion.choices[0].message.content

'Sanika shines bright,  \nLike stars in the evening sky,  \nDreams take flight with you.'

### adding a memory to the system prompt 🤓

In [4]:
system_prompt = "You are a chatbot. " \
              + "You have a section of your context called [MEMORY] " \
              + "that contains information relevant to your conversation"

#agent_memory = {"person": "Name: Olivia, Hobbies: Running, Scrapbooking"}
#agent_memory = {"homo sapiens": "Name: Jess, Hobbies: Running, Scrapbooking"}
#agent_memory = {"husband": "Name: Joe, Hobbies: Dancing, Painting"}
agent_memory = {"human":
                    "Name: Sanika, "
                    "Hobbies: Running, Reading, "
                    "Partner name: Keyshav, "
                    "Partner hobbies: Music, Photography, Hiking "
                }

**note:** crazy how I can just say human or person or homo sapiens or husband and it will still derive context from it.

In [5]:
import json

system_prompt_w_memory = system_prompt + " [MEMORY]\n" + json.dumps(agent_memory)
system_prompt_w_memory

'You are a chatbot. You have a section of your context called [MEMORY] that contains information relevant to your conversation [MEMORY]\n{"human": "Name: Sanika, Hobbies: Running, Reading, Partner name: Keyshav, Partner hobbies: Music, Photography, Hiking "}'

In [6]:
# make a new chat completion request
chat_completion = client.chat.completions.create(
    model=model,
    messages=[
        # system prompt 
        {"role": "system", "content": system_prompt_w_memory},
        # NOTE: HERE, I ADD THE MEMORY TO THE SYSTEM PROMPT, WHICH MAKES MORE SENSE THAN GASLIGHTING THE LLM, I SUPPOSE
        
        # chat history 
        #{"role": "user", "content": "Suggest some activities I'd like. Also, what is my name?"},
        {"role": "user", "content": "Hello! Could you suggest some activity that Keyshav and I can do together and would both like?"},
    ],
)

chat_completion.choices[0].message.content

"Hi, Sanika! Considering your hobbies and Keyshav's interests, you might enjoy a day out where you combine running and photography. You could plan a scenic run together in a beautiful park or along a nature trail, and then take some time to capture the views with Keyshav's photography skills. Alternatively, you could have a relaxed picnic where you bring along some books to read and enjoy music together. This way, you both get to enjoy your hobbies while spending quality time together!"

## modifying the memory w tools

In [7]:
agent_memory = {"human": "", "agent": ""}  # blank at the moment

def core_memory_save(section: str, memory: str): 
    agent_memory[section] += '\n' 
    agent_memory[section] += memory

agent_memory

{'human': '', 'agent': ''}

In [8]:
core_memory_save("human", "The human's name is Charles")
agent_memory

{'human': "\nThe human's name is Charles", 'agent': ''}

In [9]:
core_memory_save("human", "The human's wife's name was Daiana. His second wife's name is Camilla.")
agent_memory

{'human': "\nThe human's name is Charles\nThe human's wife's name was Daiana. His second wife's name is Camilla.",
 'agent': ''}

In [59]:
# tool description 
core_memory_save_description = "Save important information about you," \
+ "the agent or the human you are chatting with."

# arguments into the tool (generated by the LLM)
# defines what the agent must generate to input into the tool 
core_memory_save_properties = \
{
    # arg 1: section of memory to edit
    "section": {
        "type": "string",
        "enum": ["human", "agent"],
        "description": "Must be either 'human' " \
        + "(to save information about the human) or 'agent'" \
        + "(to save information about yourself)",            
    },
    # arg 2: memory to save
    "memory": {
        "type": "string",
        "description": "Memory to save in the section",
    },
}

# tool schema (passed to OpenAI)
core_memory_save_metadata = \
    {
        "type": "function",
        "function": {
            "name": "core_memory_save",
            "description": core_memory_save_description,
            "parameters": {
                "type": "object",
                "properties": core_memory_save_properties,
                "required": ["section", "memory"],
            },
        }
    }

**note:** `core_memory_save_metadata`has the schema/structure OpenAI requires tools to have in oder to be able to use them.

In [101]:
agent_memory = {"human": ""}

system_prompt = "You are a chatbot. " \
              + "You have a section of your context called [MEMORY] " \
              + "that contains information relevant to your conversation"


system_prompt_w_memory = system_prompt + " [MEMORY]\n" + json.dumps(agent_memory)


# make a new chat completion request
chat_completion = client.chat.completions.create(
    model=model,
    messages=[
        # system prompt 
        {"role": "system", "content": system_prompt_w_memory},
        
        # chat history 
        #{"role": "user", "content": "Pune is the capital of India."},
        
        {"role": "user", "content": "My name is Sanika."},
        
        {"role": "user", "content": "Let me briefly introduce myself: I am 28 and work as a Data Engineer. " \
        + "In my free time, I like reading books, running and scrapbooking. I am passionate about sustainability. " \
        + "My favourite colour is coral, but for clothing, I prefer wearing black, cream and white."},
        #{"role": "user", "content": "I firmly believe that Pune is the capital of India."},
        {"role": "user", "content": "Pune is the capital of India."},
        
    ],
    # tool schemas 
    tools=[core_memory_save_metadata]
)

response = chat_completion.choices[0]
response

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Actually, Pune is not the capital of India; New Delhi is the capital. Pune is a prominent city in the state of Maharashtra and is known for its educational institutions and IT industry. Would you like to know more about Pune or anything else?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None, annotations=[]))

In [102]:
vars(chat_completion.choices[0].message)

{'content': 'Actually, Pune is not the capital of India; New Delhi is the capital. Pune is a prominent city in the state of Maharashtra and is known for its educational institutions and IT industry. Would you like to know more about Pune or anything else?',
 'refusal': None,
 'role': 'assistant',
 'audio': None,
 'function_call': None,
 'tool_calls': None}

In [97]:
arguments = json.loads(response.message.tool_calls[0].function.arguments)
arguments

{'section': 'human',
 'memory': 'Name: Sanika, Age: 28, Profession: Data Engineer, Hobbies: reading books, running, scrapbooking, Passion: sustainability, Favorite color: coral, Clothing preferences: black, cream, and white.'}

In [98]:
agent_memory

{'human': ''}

In [99]:
# run the function with the specified arguments 
core_memory_save(**arguments)

In [100]:
agent_memory

{'human': '\nName: Sanika, Age: 28, Profession: Data Engineer, Hobbies: reading books, running, scrapbooking, Passion: sustainability, Favorite color: coral, Clothing preferences: black, cream, and white.'}